In [1]:
# This is a sample Python script.
import pandas as pd
import torch
device = torch.device('cpu')
import dgl
import networkx as nx
import matplotlib.pyplot as plt
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch.nn as nn
import torch.optim as optim
import dgl.function as fn


In [2]:
df=pd.read_csv("../data/PaySim_kaggle.csv")

df=df.sample(n=5000000)

print(df.head())

         step      type     amount     nameOrig  oldbalanceOrg  \
3637253   274   PAYMENT    7373.21   C881990608            0.0   
998398     45   PAYMENT    5640.53  C1286376759        32615.0   
1708338   160  TRANSFER  187660.37  C1888545418        10920.0   
2206533   186   PAYMENT    2565.17  C1626154761            0.0   
3206054   249  CASH_OUT  353256.96  C2012861900          473.0   

         newbalanceOrig     nameDest  oldbalanceDest  newbalanceDest  isFraud  \
3637253            0.00   M767846099             0.0            0.00        0   
998398         26974.47  M1564171620             0.0            0.00        0   
1708338            0.00  C1458605482             0.0       187660.37        0   
2206533            0.00   M761403458             0.0            0.00        0   
3206054            0.00  C1063995645             0.0       353256.96        0   

         isFlaggedFraud  
3637253               0  
998398                0  
1708338               0  
2206533     

In [3]:
# Set the proportion of non-fraud to remove (e.g., 50% of non-fraud instances)
remove_fraction = 0.99

# Separate the fraud and non-fraud instances
fraud_df = df[df['isFraud'] == 1]
non_fraud_df = df[df['isFraud'] == 0]

# Randomly sample and remove 'remove_fraction' proportion of non-fraud instances
non_fraud_to_remove = non_fraud_df.sample(frac=remove_fraction, random_state=42)

# Drop the sampled non-fraud instances from the DataFrame
df = df.drop(non_fraud_to_remove.index)

# Verify the new balance
label_counts = df['isFraud'].value_counts()
fraud_ratio = label_counts[1] / len(df)
non_fraud_ratio = label_counts[0] / len(df)

print(f"Fraud count: {label_counts[1]}")
print(f"Non-fraud count: {label_counts[0]}")
print(f"Fraud ratio: {fraud_ratio:.4f}")
print(f"Non-fraud ratio: {non_fraud_ratio:.4f}")


Fraud count: 6409
Non-fraud count: 49936
Fraud ratio: 0.1137
Non-fraud ratio: 0.8863


In [4]:
# Create a mapping from unique user names to numeric IDs (nodes)
user_mapping = {user: idx for idx, user in enumerate(set(df['nameOrig']).union(set(df['nameDest'])))}

# Create edges between nameOrig and nameDest
src = df['nameOrig'].map(user_mapping).values
dst = df['nameDest'].map(user_mapping).values

In [5]:
print(src)

[ 98951  16608  43525 ... 109646  39258  29190]


In [6]:
# Create a DGL graph from the source and destination nodes
g = dgl.graph((src, dst))


In [7]:
# Add transaction amount as edge feature
g.edata['amount'] = torch.tensor(df['amount'].values, dtype=torch.float32)

# Optional: Add fraud information to edge features
g.edata['isFraud'] = torch.tensor(df['isFraud'].values, dtype=torch.float32)

# Initialize node features with zeros (this handles all nodes)
num_nodes = g.num_nodes()
balance_orig = torch.zeros(num_nodes, dtype=torch.float32)
balance_dest = torch.zeros(num_nodes, dtype=torch.float32)

for orig_user, balance in df[['nameOrig', 'oldbalanceOrg']].drop_duplicates().values:
    balance_orig[user_mapping[orig_user]] = balance

for dest_user, balance in df[['nameDest', 'oldbalanceDest']].drop_duplicates().values:
    balance_dest[user_mapping[dest_user]] = balance

node_features = torch.stack([balance_orig, balance_dest], dim=1)  # Changed to stack both features

g.ndata['features'] = node_features

node_labels = torch.zeros(num_nodes, dtype=torch.float32)

# Map 'isFraud' values to corresponding nodes based on user mapping
for user, fraud in df[['nameOrig', 'isFraud']].drop_duplicates().values:
    node_labels[user_mapping[user]] = fraud


# Map 'isFraud' to destination nodes (nameDest)
for dest_user, fraud in df[['nameDest', 'isFraud']].drop_duplicates().values:
    node_labels[user_mapping[dest_user]] = fraud

# Store the fraud labels in g.ndata['isFraud']
g.ndata['isFraud'] = node_labels


In [8]:
g.edata['isFraud'].shape
#node_features.shape
g.num_edges

<bound method DGLGraph.num_edges of Graph(num_nodes=110511, num_edges=56345,
      ndata_schemes={'features': Scheme(shape=(2,), dtype=torch.float32), 'isFraud': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'amount': Scheme(shape=(), dtype=torch.float32), 'isFraud': Scheme(shape=(), dtype=torch.float32)})>

In [9]:
# Print graph information
#print(g)

# Visualize the graph using NetworkX (convert DGL graph to NetworkX for visualization)
#nx_graph = g.to_networkx()

# Optional: Visualize using a layout for better readability
#pos = nx.spring_layout(nx_graph)  # Use a layout for better visualization
#plt.figure(figsize=(12, 12))
#nx.draw(nx_graph, pos, node_size=50, node_color='skyblue', font_size=10, with_labels=True)
#plt.show()

In [10]:
# Assuming that you have edge features like transaction amounts or fraud status
edge_features = g.edata.get('amount', None)  # Assuming 'amount' is an edge feature
if edge_features is not None:
    print("Edge Features (Transaction Amounts):")
    print(edge_features)
else:
    print("No edge features found.")


Edge Features (Transaction Amounts):
tensor([211672.7031,   2996.7200,  27319.2207,  ..., 179439.6719,
          3330.6799,  29678.9805])


In [11]:
# Select 80% of nodes for training
num_train_nodes = int(0.8 * num_nodes)
train_indices = torch.randperm(num_nodes)[:num_train_nodes]  
test_indices = torch.tensor([i for i in range(num_nodes) if i not in train_indices])

# Create train/test masks
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[train_indices] = True
test_mask[test_indices] = True

# Assign to graph
g.ndata['train_mask'] = train_mask
g.ndata['test_mask'] = test_mask


In [12]:
# Define the GraphSAGE model for fraud detection
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super(GraphSAGE, self).__init__()
        self.layer1 = dgl.nn.SAGEConv(in_feats, hidden_feats, 'mean')
        self.layer2 = dgl.nn.SAGEConv(hidden_feats, out_feats, 'mean')
        self.dropout = nn.Dropout(0.5)
        #self.fc = nn.Linear(out_feats * 2, 1)  # * Concatenate source and destination node embeddings *
        self.fc = nn.Linear(out_feats, 1)  # Output a single value per node (fraud score)
        
    def forward(self, g, features):
        # Apply first GraphSAGE layer and ReLU
        x = self.layer1(g, features)
        x = torch.relu(x)
        x = self.dropout(x)

        # Apply second GraphSAGE layer to get node embeddings
        x = self.layer2(g, x)
        
        # Output a prediction for each node
        logits = self.fc(x).squeeze()  # Output a single value per node
        return logits
        



In [13]:
# Count occurrences of each label in the 'isFraud' column
label_counts = df['isFraud'].value_counts()

# Calculate the proportion of each class
fraud_ratio = label_counts[1] / len(df)  # Assuming '1' represents fraud
non_fraud_ratio = label_counts[0] / len(df)  # Assuming '0' represents non-fraud

# Print the results
print(f"Fraud count: {label_counts[1]}")
print(f"Non-fraud count: {label_counts[0]}")
print(f"Fraud ratio: {fraud_ratio:.4f}")
print(f"Non-fraud ratio: {non_fraud_ratio:.4f}")

Fraud count: 6409
Non-fraud count: 49936
Fraud ratio: 0.1137
Non-fraud ratio: 0.8863


In [23]:
# Instantiate the model
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
in_feats = 2  # balanceOrig and balanceDest features
hidden_feats = 64
out_feats = 1  # Fraud (binary classification)

pos_weight = torch.tensor([non_fraud_ratio/fraud_ratio],dtype=torch.float)

model = GraphSAGE(in_feats, hidden_feats, out_feats)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
epochs = 100
for epoch in range(epochs):
    model.train()

    # Forward pass
    logits = model(g, g.ndata['features'])
    #print(logits.shape)

    # Get target labels for the fraud detection task
    labels = g.ndata['isFraud']
    breakpoint()
    # Compute loss (use train_mask to filter out test edges)
    #print("labels",labels[train_mask].view(-1,1).shape)
    #print("logits", logits[train_mask].view(-1,1).shape)
    loss = loss_fn(logits[train_mask].view(-1,1), labels[train_mask].view(-1,1))
    
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss every few epochs
    if (epoch + 1) % 2 == 0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")



Epoch 2/100, Loss: 44732.2421875
Epoch 4/100, Loss: 28791.125
Epoch 6/100, Loss: 22544.580078125
Epoch 8/100, Loss: 16632.666015625
Epoch 10/100, Loss: 8951.3779296875
Epoch 12/100, Loss: 2283.446044921875
Epoch 14/100, Loss: 5016.560546875
Epoch 16/100, Loss: 6326.47314453125
Epoch 18/100, Loss: 4026.737060546875
Epoch 20/100, Loss: 314.09515380859375
Epoch 22/100, Loss: 2020.7969970703125
Epoch 24/100, Loss: 3303.24072265625
Epoch 26/100, Loss: 3292.263427734375
Epoch 28/100, Loss: 2442.7509765625
Epoch 30/100, Loss: 1223.515380859375
Epoch 32/100, Loss: 246.31993103027344
Epoch 34/100, Loss: 1165.792236328125
Epoch 36/100, Loss: 1140.3226318359375
Epoch 38/100, Loss: 343.8805236816406
Epoch 40/100, Loss: 522.3054809570312
Epoch 42/100, Loss: 696.5490112304688
Epoch 44/100, Loss: 395.1559143066406
Epoch 46/100, Loss: 388.766845703125
Epoch 48/100, Loss: 692.5899658203125
Epoch 50/100, Loss: 191.53501892089844
Epoch 52/100, Loss: 447.5279541015625
Epoch 54/100, Loss: 523.6150512695312

Epoch 2/20, Loss: 50026236.0
Epoch 4/20, Loss: 21910472.0
Epoch 6/20, Loss: 12671362.0
Epoch 8/20, Loss: 12492275.0
Epoch 10/20, Loss: 13373789.0
Epoch 12/20, Loss: 14267830.0
Epoch 14/20, Loss: 14137046.0
Epoch 16/20, Loss: 12949893.0
Epoch 18/20, Loss: 11810090.0
Epoch 20/20, Loss: 10091464.0
0.1
Accuracy: 0.7332
Recall: 0.0086
F1 Score: 0.0073
0.01
Accuracy: 0.7535
Recall: 0.0039
F1 Score: 0.0036
0.2
Accuracy: 0.7378
Recall: 0.0102
F1 Score: 0.0087
0.4
Accuracy: 0.7405
Recall: 0.0079
F1 Score: 0.0068

In [16]:
g.ndata.keys()

dict_keys(['features', 'isFraud', 'train_mask', 'test_mask'])

In [24]:


# Evaluate the model
model.eval()
with torch.no_grad():
    logits = model(g, g.ndata['features'])

    # Convert logits to probabilities
    predictions = torch.sigmoid(logits).squeeze()
    
    # Apply threshold of 0.5 to classify fraud
    predicted_labels = (predictions > 0.5).float()
    predicted_labels = predicted_labels[test_mask]  # Apply test_mask here
    # Get actual labels
    #true_labels = g.edata['isFraud']
    true_labels = g.ndata['isFraud'][test_mask]  # Apply test_mask here
    # Compute accuracy manually
    correct = (predicted_labels == true_labels).sum().item()
    total = true_labels.size(0)
    accuracy = correct / total

    # Print evaluation results
    print(f"Accuracy: {accuracy:.4f}")
    # Calculate F1-Score
    f1 = f1_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    print(f"Recall: {recall:.4f}")
    print(f"Precision:{precision:.4f}")
    print(f"F1 Score: {f1:.4f}")

Accuracy: 0.6761
Recall: 0.3880
Precision:0.1475
F1 Score: 0.2137


In [18]:
g.ndata['features']
predicted_labels

tensor([1., 1., 1.,  ..., 0., 1., 0.])

In [19]:
sum(labels==1)

tensor(12639)